In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

If launching Tensorboards throws an error, run the diagnosis script:
```
python tensorboard_diagnosis.py
```

### Extract validation images from TensorBoard event file

Source: https://gist.github.com/hysts/81a0d30ac4f33dfa0c8859383aec42c2

In [2]:
import numpy as np
import cv2
import pathlib
import os
from tensorboard.backend.event_processing import event_accumulator


def extract_val_images(path, outdir):

    event_acc = event_accumulator.EventAccumulator(
        path, size_guidance={'images': 0})
    event_acc.Reload()

    outdir = pathlib.Path(outdir)
    outdir.mkdir(exist_ok=True, parents=True)

    for tag in event_acc.Tags()['images']:
        events = event_acc.Images(tag)

        tag_name = tag.replace('/', '_')
        dirpath = outdir / tag_name
        dirpath.mkdir(exist_ok=True, parents=True)

        for index, event in enumerate(events):
            s = np.frombuffer(event.encoded_image_string, dtype=np.uint8)
            image = cv2.imdecode(s, cv2.IMREAD_COLOR)
            outpath = dirpath / '{:04}.jpg'.format(index)
            cv2.imwrite(outpath.as_posix(), image)

In [4]:
TRAIN_OUT_DIR = "C:\\Users\\viscom\\workspace\\tree_project\\Training_Output"
path_event_log = os.path.join(TRAIN_OUT_DIR, "cn1", "logs", "controlnet_trees")
out_dir =  os.path.join(TRAIN_OUT_DIR, "cn1", "val_images")
extract_val_images(path_event_log, out_dir)